# Dashboard for T&T

## 1. Import & set up

### 1.1 Packages

In [21]:
import time  # to simulate a real time data, time loop
import numpy as np  # np mean, np random
import pandas as pd  # read csv, df manipulation
import plotly.express as px  # interactive charts
import pickle #export

### 1.2 Datasets: All of the file locations must be updated based on your local computer / cloud

In [2]:
fields=['Month', 'Cancelled', 'Operating_Airline ', 'TaxiOut', 'DepDelay', 'Distance', 'DestStateName']
january=pd.read_csv('/content/drive/MyDrive/Dashboard_dataset/january.csv', usecols=fields)
february=pd.read_csv('/content/drive/MyDrive/Dashboard_dataset/february.csv', usecols=fields)
march=pd.read_csv('/content/drive/MyDrive/Dashboard_dataset/march.csv', usecols=fields)
april=pd.read_csv('/content/drive/MyDrive/Dashboard_dataset/april.csv', usecols=fields)
t100=pd.read_csv('/content/drive/MyDrive/Dashboard_dataset/t100_2023.csv')

All datasets are concatenated into a single dataframe

In [3]:
complete=pd.concat([january, february, march, april])
complete.head()

,Month,Operating_Airline,DestStateName,DepDelay,TaxiOut,Cancelled,Distance
0,1,B6,Florida,-4.0,12.0,0.0,1032.0
1,1,B6,Florida,53.0,16.0,0.0,1121.0
2,1,B6,Puerto Rico,7.0,25.0,0.0,1046.0
3,1,B6,Florida,7.0,15.0,0.0,1046.0
4,1,B6,Virginia,-8.0,16.0,0.0,399.0


A dictionary is defined to replace coded carrier names with common names

In [4]:
operators={"9E":"Endeavor",
"AA":	"American",
"AS":	"Alaska",
"B6":	"JetBlue",
"C5":	"CommuteAir",
"DL":	"Delta",
"F9":	"Frontier",
"G4":	"Allegiant",
"G7":	"GoJet",
"HA":	"Hawaiian",
"MQ":	"Envoy",
"NK":	"Spirit",
"OH":	"PSA",
"OO":	"SkyWest",
"PT":	"Piedmont",
"QX":	"Horizon",
"UA":	"United",
"WN":	"Southwest",
"YV":	"Mesa",
"YX":	"Republic",
"ZW":	"Air Wisconsin"}

The dictionary is then used to replace the names in the corresponding column of the dataframe

In [5]:
complete=complete.replace({'Operating_Airline ':operators})
carrierp=pd.DataFrame(complete['Operating_Airline '].value_counts().rename_axis('Carrier'))
carrierp.rename(columns={"Operating_Airline ": 'Counts'}, inplace=True)

## 2. Data processing

### 2.1. Cancelled flights data, currently defined for 4 months

In [14]:
cancelled_series=complete['Cancelled'].groupby(complete.Month).agg('sum')
cancelled = pd.DataFrame({
  'date': ['1/23','2/23', '3/23', '4/23'],
  'Total': cancelled_series
})
for op in operators.values():
  isolate=complete[complete['Operating_Airline ']==op]
  new=isolate.groupby(isolate.Month).agg('sum')['Cancelled']
  cancelled[op]=new
cancelled = cancelled.set_index('date')

<ipython-input-14-f7580c4ac441>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new=isolate.groupby(isolate.Month).agg('sum')['Cancelled']
<ipython-input-14-f7580c4ac441>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new=isolate.groupby(isolate.Month).agg('sum')['Cancelled']
<ipython-input-14-f7580c4ac441>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  new=isolate.groupby(isolate.Month).agg('sum')['Cancelled']
<ipy

In [17]:
cancelled.head()

,Total,Endeavor,American,Alaska,JetBlue,CommuteAir,Delta,Frontier,Allegiant,GoJet,...,Spirit,PSA,SkyWest,Piedmont,Horizon,United,Southwest,Mesa,Republic,Air Wisconsin
date,,,,,,,,,,,,,,,,,,,,,
1/23,11032.0,249.0,1417.0,280.0,194.0,252.0,586.0,438.0,115.0,43.0,...,507.0,250.0,1666.0,111.0,70.0,415.0,3234.0,175.0,386.0,104.0
2/23,9686.0,356.0,1447.0,573.0,372.0,84.0,968.0,185.0,58.0,90.0,...,272.0,47.0,1351.0,59.0,103.0,398.0,2143.0,204.0,388.0,127.0
3/23,7847.0,232.0,859.0,204.0,403.0,52.0,1589.0,211.0,94.0,97.0,...,381.0,176.0,972.0,99.0,19.0,403.0,1116.0,123.0,440.0,65.0
4/23,10323.0,113.0,1235.0,220.0,705.0,118.0,1837.0,214.0,139.0,180.0,...,815.0,150.0,808.0,38.0,51.0,953.0,1249.0,222.0,688.0,129.0


### 2.2 PAX and cargo data, currently defined for 5 months

In [18]:
paxcargo_series=t100.groupby(t100.MONTH).agg('sum')
paxcargo = pd.DataFrame({
  'date': ['1/23','2/23', '3/23', '4/23', '5/23'],
  'PAX Volume (10⁷)': paxcargo_series['PASSENGERS']/(10**7),
  'Cargo Volume (10⁹)': paxcargo_series['FREIGHT']/(10**9)
})
paxcargo = paxcargo.set_index('date')
paxcargo

<ipython-input-18-35423b63b55a>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  paxcargo_series=t100.groupby(t100.MONTH).agg('sum')


,PAX Volume (10⁷),Cargo Volume (10⁹)
date,,
1/23,6.810921,2.770908
2/23,6.548552,2.679373
3/23,8.068277,3.217284
4/23,7.847479,2.934295
5/23,8.263561,3.060820


### 2.3 Geographical data, used for the map on the dashboard

In [20]:
bystate=complete.groupby(['DestStateName','Month'])['Cancelled'].agg('sum')
bystate = pd.DataFrame({
  'Total': bystate
})
bystate.reset_index(inplace=True)
bystate = bystate.rename(columns = {'index':'State Name'})
bystate.head()

,DestStateName,Month,Total
0,Alabama,1,42.0
1,Alabama,2,28.0
2,Alabama,3,29.0
3,Alabama,4,42.0
4,Alaska,1,69.0


## 3. Data export

All values are stored in a txt file named "sample.txt"

In [22]:
total = cancelled[['Total']]
airlines=cancelled[['Southwest','Delta', 'American']]
with open('sample.txt', 'wb') as f:
    pickle.dump([total, airlines, complete.sample(n=1000, random_state=1), bystate, carrierp, paxcargo],f)

Test to verify values stored in the data file

In [23]:
with open('sample.txt', 'rb') as f:
    print(pickle.load(f))

[        Total
date         
1/23  11032.0
2/23   9686.0
3/23   7847.0
4/23  10323.0,       Southwest   Delta  American
date                             
1/23     3234.0   586.0    1417.0
2/23     2143.0   968.0    1447.0
3/23     1116.0  1589.0     859.0
4/23     1249.0  1837.0    1235.0,         Month Operating_Airline  DestStateName  DepDelay  TaxiOut  Cancelled  \
33952       4          Southwest     Tennessee     102.0     13.0        0.0   
450195      4          Allegiant       Florida      17.0     10.0        0.0   
190253      1           American         Texas       0.0     15.0        0.0   
263418      4          Southwest       Florida       7.0     15.0        0.0   
90094       4            Horizon    California     -10.0      8.0        0.0   
...       ...                ...           ...       ...      ...        ...   
21842       4          Southwest      Illinois      10.0      9.0        0.0   
473197      3              Delta       Florida      -4.0     19.0    

## (OBSOLETE) Manage tunneling

In [ ]:
!pip install ngrok

In [ ]:
from pyngrok import ngrok
ngrok.disconnect(8501)
ngrok.set_auth_token("")
public_url = ngrok.connect(8501)
print(public_url)

http://0074-34-82-31-41.ngrok-free.app


## (REFERENCE) Logan files

In [ ]:
!pip install streamlit-folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import pandas as pd
import folium
import geopandas as gpd
from folium.features import GeoJsonPopup, GeoJsonTooltip
import streamlit as st
from streamlit_folium import folium_static
import json

@st.cache
def read_csv(path):
    return pd.read_csv(path, compression='gzip', sep='\t', quotechar='"')

#Data prepartion to only retrieve fields that are relevent to this project
data_df = pd.read_csv('test_chlorpleth_data.csv')
data_df['Focus CSD_ID']=data_df['Focus CSD_ID'].astype(str)

"""
@st.cache
def read_file(path):
    return gpd.read_file(path)
"""

#Read the geojson file
gdf = gpd.read_file('NH_CSDs.geojson')
data_df['Focus CSD_ID']=data_df['Focus CSD_ID'].astype(str)

#st.write(gdf.head())
#type(data_df['Focus CSD_ID'][1])

#Merge the housing market data and geojson file into one dataframe
df_final = gdf.merge(data_df, left_on="GEOID", right_on="Focus CSD_ID", how="outer")

#st.write(df_final.head())

#Add sidebar to the app
st.sidebar.markdown("### My first Awesome App")
st.sidebar.markdown("Welcome to my first awesome app. This app is built using Streamlit and uses data source from redfin housing market data. I hope you enjoy!")

#Add title and subtitle to the main interface of the app
st.title("County Subdivision Migration Visualization")
st.markdown("Test to visualize net migration into NH Hillsborough county subdivisions")

#Initiate a folium map
m = folium.Map(location=[40, -100], zoom_start=4,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

#Plot Choropleth map using folium
choropleth1 = folium.Choropleth(
    geo_data= 'NH_CSDs.geojson',#.../us-state-boundaries.geojson',     #This is the geojson file for the Unite States
    name='Choropleth Map of U.S. Housing Prices',
    data=data_df,                                  #This is the dataframe we created in the data preparation step
    columns=['Focus CSD_ID', 'Moved NET'],                #'state code' and 'metrics' are the two columns in the dataframe that we use to grab the median sales price for each state and plot it in the choropleth map
    key_on='feature.properties.GEOID',             #This is the key in the geojson file that we use to grab the geometries for each state in order to add the geographical boundary layers to the map
    fill_color='YlGn',
    nan_fill_color="White",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Housing Market Metrics',
    highlight=True,
    line_color='black').geojson.add_to(m)

folium_static(m)

Overwriting app.py


In [ ]:
%%writefile app.py

import pandas as pd
import numpy as np
import streamlit as st


@st.cache
def read_csv(path):
    return pd.read_csv(path, compression='gzip', sep='\t', quotechar='"')

# set tabs
tab1, tab2, tab3, tab4, tab5 = st.tabs(["Performance", "Carriers", "Flights", "Fares", "Cargo"])

# PERFORMANCE TAB =============================================================
with tab1:
   st.header("A cat")
   st.image("https://static.streamlit.io/examples/cat.jpg", width=200)

   # Add a selectbox to the sidebar:
   add_selectbox = st.sidebar.selectbox(
        'How would you like to be contacted?',
        ('Email', 'Home phone', 'Mobile phone')
   )

   # Add a slider to the sidebar:
   add_slider = st.sidebar.slider(
    'Select a range of values',
    0, 100, (25, 75)
   )


with tab2:
   st.header("A dog")
   st.image("https://static.streamlit.io/examples/dog.jpg", width=200)

with tab3:
   st.header("An owl")
   st.image("https://static.streamlit.io/examples/owl.jpg", width=200)

Overwriting app.py
